# Lecture 4
We will now look at ways to run a computational process and monitor it as it performs the assigned task. We will start with just running simple Python functions that we write, but we will prepare to work with a large scale program.

To do that we will:
* need to read and modify a setting file
* read and monitor some value
* spawn a process and stop it if need be